In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn.functional as nnF
import torchvision
from torchvision import datasets, models, transforms
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
import numpy as np
import os
import time
import copy
import glob
import json
from PIL import Image
import matplotlib.pyplot as plt
import cv2
import pandas as pd
import dill
## This allows inbuilt cudnn auto-tuner to find the best algorithm to use for your hardware
import torch.backends.cudnn as cudnn
cudnn.benchmark = True

In [ ]:
'''
# For google drive connection in order to load the dataset
from google.colab import drive
drive.mount('/content/drive')

datasetRootPath = 'drive/MyDrive/Colab Notebooks/data/BreaKHis_v1'
'''

# For google drive connection in order to load the dataset
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

!ls drive/MyDrive/mock_pathology_GAN_transformed_img/*
# To use stain normalized images
#datasetRootPath = '/media/wenrui/c5fdcf91-411a-4936-a09e-8e6585f9fe28/data/BreakHis/breakhis-analysis/400xData_normalized'

# To use non-normalized images
#datasetRootPath = "/media/wenrui/c5fdcf91-411a-4936-a09e-8e6585f9fe28/data/BreakHis/image_data/breast-cancer-ml-reu/data/BreaKHis_v1/BreaKHis_v1/histology_slides/breast/"

datasetRootPath= "drive/MyDrive/mock_pathology_GAN_transformed_img"

# Global Configuration

In [ ]:
## General parameters
verbose = False
check = False

fileName_BM     = ['_B_', '_M_']
path_BM         = ['benign', 'malignant']
magnifications  = ['40X', '100X', '200X', '400X']

## Enumerates and dictionaries
folds = ['fold-0', 'fold-1', 'fold-2', 'fold-3', 'fold-4']
class_names = [ 'Benign', 'Malignant']

## Execution parameters
num_epochs = 30     # Epochs to be trained
#lr=0.001            # Learning rate for the training
lr=1e-3
#momentum=0.9        # Momentum for the training
returned='best'     # Model to be returned in the training. ['last' (default), 'best']


#add device to cpu
#device = torch.device("cpu")

# Dataloader

In [ ]:
class MyDataset(Dataset):
    def __init__(self, data, targets, names):
        self.data=data
        self.targets=targets
        self.names=names

    def __len__(self):
        #return the number of data points
        return self.data.shape[0]

    def __getitem__(self, idx):
        x = self.data[idx]
        y = torch.tensor(self.targets[idx], dtype=torch.float32)
        z = self.names[idx]
        return x, y, z

class DatasetDataContainer:
    data = []
    targets = []
    names = []

    def __init__(self, data, targets, names, test_size=0.1, k_splits=5, random_state=42, transformations=None):
        self.data = data
        self.targets = targets
        self.names = names
        self.k_splits = k_splits
        self.random_state = random_state
        self.transformations = transformations
        if self.transformations is None:
            self.transformations = transforms.ToTensor()

        # Generate train-test indexes
        sss = StratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state=self.random_state)
        self.train_index, self.test_index = list(sss.split(self.data, self.targets, self.names))[0]
        
        # Generate train-validation k-fold indexes
        strtfdKFold = StratifiedKFold(n_splits=self.k_splits, shuffle=True, random_state=self.random_state)
        self.kfold = list(strtfdKFold.split(self.data[self.train_index], self.targets[self.train_index], self.names[self.train_index]))
    
    def get_fold (self, fold, transformations):
        train, val = self.kfold[fold]
        fold_train = self.train_index[train]
        fold_val = self.train_index[val]
        #np.asarray(self.transformations(Image.open(fpath))) 
        train_dataset = MyDataset(np.asarray([np.asarray(transformations[0](x)) for x in self.data[fold_train]]), self.targets[fold_train], self.names[fold_train])
        val_dataset = MyDataset(np.asarray([np.asarray(transformations[1](x)) for x in self.data[fold_val]]), self.targets[fold_val], self.names[fold_val])
        return train_dataset, val_dataset

    def get_fold_dataloaders(self, fold, transformations, batch_size=32, shuffle=True, num_workers=2):
        train_dataset, val_dataset = self.get_fold(fold, transformations)
        test_dataset = MyDataset(np.asarray([np.asarray(transformations[1](x)) for x in self.data[self.test_index]]), self.targets[self.test_index], self.names[self.test_index])
        train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)
        val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)
        test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)
        return {'train': train_dataloader, 'val': val_dataloader, 'test': test_dataloader}


class BreakHistDataContainer:

    pathLabelsList = ['benign', 'malignant']

    def __init__(self, folderPath, magnification, test_size=0.1, k_splits=5, random_state=42, transformations=None):
        self.transformations = transformations
        if self.transformations is None:
            self.transformations = transforms.ToTensor()
        data, labels, names = self.read_images(folderPath, magnification)
        self.dataContainer = DatasetDataContainer (data, labels, names, test_size, k_splits, random_state, transformations)
    
    def image_reader(self, fpath):
        img = cv2.imread(fpath)
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        #print(img.shape)
        #print(ia.is_np_array(img))
 
        return img
    
    def read_images (self, folderPath, magnification):
        
        # Take the paths of all images for that magnification in the given folder and subfolders and short them alphabetically
        filepaths = glob.glob(folderPath + "/**/*.png", recursive=True)
        filepaths = [fpath for fpath in filepaths if magnification in fpath]
        self.filepaths = sorted(filepaths, key=lambda s: os.path.split(s)[-1])

        # Read all images and extract their labels from the path
        
        # images = [np.asarray(self.transformations(Image.open(fpath))) for fpath in self.filepaths]
        images = [np.asarray(self.image_reader(fpath)) for fpath in self.filepaths]
        #images = list(self.filepaths)
        labels = [0 if self.pathLabelsList[0] in fpath else 1 for fpath in self.filepaths]
        names = [fpath.split("/")[-1] for fpath in self.filepaths]

        # Store images and labels
        data = np.asarray(images, dtype=object)
        #data = images
        labels = np.asarray(labels)
        names = np.asarray(names)

        return data, labels, names
    
    def get_fold (self, fold, transformations):
        return self.dataContainer.get_fold(fold, transformations)

    def get_fold_dataloaders(self, fold, transformations, batch_size=32, num_workers=2):
        return self.dataContainer.get_fold_dataloaders(fold, transformations, batch_size, num_workers)

# Main functions

In [ ]:
# Returns a collection of metrics for the labels and prediction given
def get_metrics (y_true, y_pred):
    tn, fp, fn, tp = metrics.confusion_matrix(y_true, y_pred).ravel()
    roc_auc = metrics.roc_auc_score(y_true, y_pred)
    acc = metrics.accuracy_score(y_true, y_pred)
    f1 = metrics.f1_score(y_true, y_pred)
    precision, recall, thresholds = metrics.precision_recall_curve(y_true, y_pred)
    pr_auc = metrics.auc(recall, precision)

    #return {'cm': [tp, fp, tn, fn], 'acc': acc, 'roc_auc': roc_auc, 'f1': f1, 'pr_auc': pr_auc}
    return {'cm': [tp, fp, tn, fn], 'acc': acc, 'roc_auc': roc_auc, 'f1': f1, 'pr_auc': pr_auc,\
           'copy to google sheet': f"{round(acc, 4)}, {round(roc_auc, 4)}, {round(f1, 4)}, {round(pr_auc, 4)},\
            {tp}, {fp}, {tn}, {fn}"
           }

# Returns an average of the metrics collection given
def average_metrics (metrics):
    tp = 0
    fn = 0
    fp = 0
    tn = 0
    acc = 0
    roc_auc = 0
    f1 = 0
    pr_auc = 0
    s = len(metrics)

    for execution in metrics:
        tp += execution['cm'][0]
        fp += execution['cm'][1]
        tn += execution['cm'][2]
        fn += execution['cm'][3]
        acc += execution['acc']
        roc_auc += execution['roc_auc']
        f1 += execution['f1']
        pr_auc += execution['pr_auc']

    return {'cm': [tp/s, fp/s, tn/s, fn/s], 'acc': acc/s, 'roc_auc': roc_auc/s, 'f1': f1/s, 'pr_auc': pr_auc/s,\
           'copy to google sheet': f"{round(acc/s, 4)}, {round(roc_auc/s, 4)}, {round(f1/s, 4)}, {round(pr_auc/s, 4)},\
            {tp/s}, {fp/s}, {tn/s}, {fn/s}"
           }

# Return a confusion matrix of the trained model received as parameter.
# dataloaders must be torch.utils.data.DataLoader
# The confusion matrix is a list with this values [tp, fp, tn, fn]
def get_evaluation_metrics (model, dataloader):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    was_training = model.training
    model.eval()

    sigm = torch.nn.Sigmoid()
    
    y_pred = []
    y_true = []

    with torch.no_grad():
        # iterate over test data
        for inputs, labels, names in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs) # Feed Network
            preds = sigm(outputs).cpu().reshape(-1).detach().numpy().round()
            
            y_pred.extend(preds) # Save Prediction
            y_true.extend(labels.cpu()) # Save Truth
    model.train(mode=was_training)
    
    return get_metrics (y_true, y_pred)

In [ ]:
def plot_epoch_evolution_ (loss_train_list, acc_train_list, acc_val_list, metric_train_list, \
                           metric_val_list, model_name, metric_name='acc', plot_size=6):
    figures= 2 if metric_name=='acc' else 3
    fig, ax = plt.subplots(1, figures, figsize=(plot_size*figures,plot_size))
    ax[0].set_title('loss v.s. epoch',fontsize=16)
    ax[0].plot(loss_train_list, '-b', label='training loss')
    ax[0].set_xlabel('epoch',fontsize=16)
    ax[0].legend(fontsize=16)
    ax[0].grid(True)
    ax[1].set_title('accuracy v.s. epoch',fontsize=16)
    ax[1].plot(acc_train_list, '-b', label='training accuracy')
    ax[1].plot(acc_val_list, '-g', label='validation accuracy')
    ax[1].set_xlabel('epoch',fontsize=16)
    ax[1].legend(fontsize=16)
    ax[1].grid(True)
    if figures==3:
        ax[2].set_title(metric_name + ' v.s. epoch',fontsize=16)
        ax[2].plot(metric_train_list, '-b', label='training ' + metric_name)
        ax[2].plot(metric_val_list, '-g', label='validation ' + metric_name)
        ax[2].set_xlabel('epoch',fontsize=16)
        ax[2].legend(fontsize=16)
        ax[2].grid(True)
    
    plt.savefig(f"{datasetRootPath}/{model_name}_epoch_curves.png")
    plt.show()

def plot_epoch_evolution (epoch_evolution_list, model_name, metric_name='acc', plot_size=6):
    loss_train_list=[]
    acc_train_list=[]
    acc_val_list=[]
    metric_train_list=[]
    metric_val_list=[]
    for (train_loss, train_metrics, val_metrics) in epoch_evolution_list:
        loss_train_list.append(train_loss)
        acc_train_list.append(train_metrics['acc'])
        acc_val_list.append(val_metrics['acc'])
        metric_train_list.append(train_metrics[metric_name])
        metric_val_list.append(val_metrics[metric_name])

    plot_epoch_evolution_ (loss_train_list, acc_train_list, acc_val_list, metric_train_list, metric_val_list,\
                           model_name, metric_name, plot_size)

In [ ]:
## Training loop of of a pytorch model
## It uses BCEWithLogitsLoss by default, so an activation functions is needed to be used in the output of the model
## If returned is 'last' by default, returns the model trained in the last epoch, but if it's 'best'
## It uses Adam optimizer by default
## the model returned is the one with the best score in validation (F1 score)
## lr and momentum are the learning rate and momentum of the scheduler
## RETURNS a tuple (metrics, model)
def train_model(model, dataloader_train, dataloader_val, model_name=None, num_epochs=25, returned='best', \
                criterion=None, optimizer=None, metric='f1', plot_evolution=True, verbose=False):
    since = time.time()

    # List to keep track of metrics evolution during training to plot at the end
    epoch_tracking_list=[]

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)

    if criterion == None: 
        #criterion = torch.nn.BCELoss()
        criterion = torch.nn.BCEWithLogitsLoss()
    if optimizer == None:
        #optimizer = optim.SGD(model.parameters())
        optimizer = optim.Adam(model.parameters(), lr=0.001)

    sigm = torch.nn.Sigmoid()

    metrics_best = {metric:0.0}
    epoch_best = 0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        if verbose:
            print('-' * 10)
        epoch_since = time.time()

        model.train()  # Set model to training mode
        running_loss = 0
        y_true = []
        y_pred = []

        # Iterate over data.
        for indx, (inputs, labels, names) in enumerate(dataloader_train):
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Transforming loop
            optimizer.zero_grad()
            with torch.set_grad_enabled(True):
                outputs  = model(inputs)
                loss = criterion(outputs, labels.float().unsqueeze(1))
                preds = sigm(outputs).cpu().reshape(-1).detach().numpy().round()

            # backward + optimize
            loss.backward()
            optimizer.step()

            # Result acumulation
            running_loss += loss.item() * inputs.size(0)
            y_pred.extend(preds) # Save Prediction
            y_true.extend(labels.cpu()) # Save Truth

        # Epoch evaluation
        train_loss = running_loss / len(y_true)
        train_metrics = get_metrics (y_true, y_pred)
        val_metrics = get_evaluation_metrics(model, dataloader_val)
        epoch_tracking_list.append((train_loss, train_metrics, val_metrics))

        if verbose: 
            epoch_time_elapsed = time.time() - epoch_since
            print('Train \tLoss: {:.4f} \t\tAcc: {:.4f}'.format(train_loss, train_metrics['acc']))
            print('Val \tCM: {} \tAcc: {:.4f} \tROC auc: {:.4f} \tF1: {:.4f} \tP/R auc: {:.4f}'.format(val_metrics['cm'], val_metrics['acc'], val_metrics['roc_auc'], val_metrics['f1'], val_metrics['pr_auc']))
            print('Time: {:.0f}m {:.0f}s'.format(epoch_time_elapsed // 60, epoch_time_elapsed % 60))
            print('')

        # deep copy the model
        if val_metrics[metric] > metrics_best[metric] and returned == 'best':
            metrics_best = val_metrics
            model_best = copy.deepcopy(model)
            epoch_best = epoch

    if returned == 'best':
        val_metrics = metrics_best
        model = model_best
        epoch = epoch_best
            
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s. Epoch returned: {} ({})'.format(time_elapsed // 60, time_elapsed % 60, epoch, returned))
    print(val_metrics)
    
    if plot_evolution:
        print('\n\n')
        plot_epoch_evolution (epoch_tracking_list, model_name, metric)

    return (val_metrics, model)

In [ ]:

def normalize_color_image(I):
    I_max = I.max(axis=(0,1), keepdims=True)
    I_min = I.min(axis=(0,1), keepdims=True)    
    I = (I - I_min)/(I_max-I_min)
    return I

def explaination_single_models (img, label, modelList):

    numImages = len(modelList)+1
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print('abs(dL/dx)')

    #--------------------------------------------------
    ffig, ax = plt.subplots(1, 2, figsize=(12,10))
    x=img.to(device)
    ax[0].imshow(x.detach().cpu().numpy().squeeze().transpose(1,2,0))
    ax[0].set_title('input image x, label='+str(label), fontsize=16)
    #--------------------------------------------------
    x=img.to(device)
    x.requires_grad=True
    z=model(x).view(1)
    y=torch.tensor([label], dtype=x.dtype, device=device)
    loss = nnF.binary_cross_entropy_with_logits(z, y)
    loss.backward()
    #--------------------------------------------------
    y=y.item()
    xx = x.detach().cpu().numpy().squeeze()
    xx=xx.transpose(1,2,0)
    x_grad=x.grad.data.detach().cpu().numpy().squeeze()
    x_grad=x_grad.transpose(1,2,0)
    x_grad=np.abs(x_grad).sum(axis=2)
    xx = normalize_color_image(xx)
    #--------------------------------------------------
    ax[1].imshow(x_grad, cmap='gray', vmin=x_grad.min(), vmax=x_grad.max())
    ax[1].set_title('abs(dL/dx)', fontsize=16)

def explaination_multiple_models (img, label, modelList):

    numImages = len(modelList)+1
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print('abs(dL/dx)')

    #--------------------------------------------------
    fig, ax = plt.subplots(1, numImages, figsize=(24,20))
    x=img.to(device)
    ax[0].imshow(x.detach().cpu().numpy().squeeze().transpose(1,2,0))
    ax[0].set_title('input image x, label='+str(label), fontsize=16)

    i = 1
    for (name, model) in modelList:
        x=img.to(device)
        x.requires_grad=True
        z=model(x).view(1)
        prediction = (z.data > 0).to(torch.int64).item()
        y=torch.tensor([label], dtype=x.dtype, device=device)
        loss = nnF.binary_cross_entropy_with_logits(z, y)
        loss.backward()
        #--------------------------------------------------
        y=y.item()
        xx = x.detach().cpu().numpy().squeeze()
        xx=xx.transpose(1,2,0)
        x_grad=x.grad.data.detach().cpu().numpy().squeeze()
        x_grad=x_grad.transpose(1,2,0)
        x_grad=np.abs(x_grad).sum(axis=2)
        xx = normalize_color_image(xx)
        ax[i].imshow(x_grad, cmap='gray', vmin=x_grad.min(), vmax=x_grad.max())
        ax[i].set_title(name + ', pred='+str(prediction), fontsize=16)
        #--------------------------------------------------
        i+=1

# Models

## IBN-ResNet / ResNet - get pretrained model

In [ ]:
# Pretrained weights for all variation of IBN-Net can be found here https://github.com/XingangPan/IBN-Net/releases/tag/v1.0

def get_pretrained_fc_linear (model_name, model_url=None, freeze = False):

    if "_ibn_" not in model_name and model_url==None:
      resname = model_name.split("_only")[0]
      weight_name = resname.replace("resnet", "ResNet")
      model = eval(f"models.{resname}(weights = torchvision.models.{weight_name}_Weights.IMAGENET1K_V1)")

    else:
      model = torch.hub.load('XingangPan/IBN-Net', model_name, pretrained=False)
      
      if freeze: # use pretrained weights
          checkpoint = torch.hub.load_state_dict_from_url(model_url)#, \
                                                          #map_location='cpu')
          # load weights
          model.load_state_dict(checkpoint)

     num_ftrs = model.fc.in_features
     model.fc = nn.Sequential (    
         nn.Linear(num_ftrs, 1)
     )

    return model


# Execution

### Data will be only initialize (randomized) once to make sure that all the model using the same split

In [ ]:
data = BreakHistDataContainer(datasetRootPath, '400X')

In [ ]:


'''
## Transformations, usually model-dependentant
crop_size = 460
input_shape = 224
#mean = [0.485, 0.456, 0.406]
#std = [0.229, 0.224, 0.225]
original data transformation
data_transforms = transforms.Compose([
        transforms.CenterCrop(crop_size),
        transforms.Resize(input_shape),
        transforms.ToTensor(),
        # transforms.Normalize(mean, std)
    ])
'''
# using new data tranform by WH
#! pip install imgaug



normalize = transforms.Normalize(mean=[0.697759863, 0.68592817, 0.582152582], std=[0.095469999, 0.102921345, 0.139130713])
mean = np.asarray([0.697759863, 0.68592817, 0.582152582])
std = np.asarray([0.095469999, 0.102921345, 0.139130713])

from imgaug import augmenters as iaa
iaa_state = 123

'''
Perform augmentation on training set only if there is augmentation
Thus, we need two separate transformers for training and val-test datasets respectively
'''

train_transforms = transforms.Compose([

        
    iaa.Sequential([iaa.size.Resize(224), iaa.SomeOf((3,7), [
    iaa.Fliplr(0.5, random_state=iaa_state),
    #iaa.CoarseDropout(0.1, size_percent=0.2, random_state=iaa_state),
    iaa.Flipud(0.5, random_state=iaa_state),
    iaa.OneOf([iaa.Affine(rotate=90, random_state=iaa_state),
                iaa.Affine(rotate=180, random_state=iaa_state),
                iaa.Affine(rotate=270, random_state=iaa_state)], random_state=iaa_state)])], random_order=True).augment_image,
    #iaa.Multiply((0.8, 1.5), random_state=iaa_state),
    #iaa.AdditiveGaussianNoise(scale=(0,0.2*255), per_channel=True, random_state=iaa_state),
    #iaa.MultiplyHue((0.94,1.04), random_state=iaa_state),
    #iaa.AddToBrightness((-20,64), random_state=iaa_state),
    #iaa.MultiplySaturation((0.75,1.25), random_state=iaa_state),
    #iaa.LinearContrast((0.25,1.75), random_state=iaa_state)])], random_order=True).augment_image,
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
    
    ])

'''
NEED TO COMMENT OUT train_transforms = test_val_transforms PART IF AUG IS NEEDED!!!! 
'''


test_val_transforms = transforms.Compose([

        
    iaa.Sequential([iaa.size.Resize(224)]).augment_image,
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
    
    ])



#train_transforms = test_val_transforms

#augmentation = 

#augmentation_valtest = iaa.Sequential([iaa.size.Resize(224)])

#augmentation_valtest.augment_images(dataloaders['test'])






In [ ]:
dataloaders = data.get_fold_dataloaders(0, [train_transforms, test_val_transforms], batch_size=32)

### Save and load dataloaders to make sure that all the training are done with exact the same dataset

In [ ]:
'''
with open(f"012323_NO_AUG_train_val_and_test_dataloaders_dill.pkl", "wb") as f:
    dill.dump(dataloaders, f, recurse=True)
    
print("Finished saving the dataloaders!")

with open(f"012323_FLIP_ONLY_train_val_and_test_dataloaders_dill.pkl", "wb") as f:
    dill.dump(dataloaders, f, recurse=True)
    
print("Finished saving the dataloaders!")
'''
#with open(f"012323_train_val_and_test_dataloaders_dill.pkl", "rb") as fp:
    #dataloaders=dill.load(fp)
    
#print("Finished loading the dataloaders!")

In [ ]:
modelList = []
metricsList = []


'''
IBN_model_nameList = ["resnet18_ibn_b_FLIP_ROTAT_ONLY", "resnet18_ibn_a_FLIP_ROTAT_ONLY",\
                 "resnet34_ibn_b_FLIP_ROTAT_ONLY", "resnet34_ibn_a_FLIP_ROTAT_ONLY",\
                  "resnet18_ibn_b_NO_AUG", "resnet18_ibn_a_NO_AUG",\
                  "resnet34_ibn_b_NO_AUG", "resnet34_ibn_a_NO_AUG",\
                  "resnet18_only_FLIP_ROTAT_ONLY", "resnet18_only_NO_AUG",\
                 "resnet34_only_FLIP_ROTAT_ONLY", "resnet34_only_NO_AUG",
                 "resnet50_ibn_b_FLIP_ROTAT_ONLY", "resnet50_ibn_a_FLIP_ROTAT_ONLY",\
                 "resnet101_ibn_b_FLIP_ROTAT_ONLY", "resnet101_ibn_a_FLIP_ROTAT_ONLY",\
                  "resnet50_ibn_b_NO_AUG", "resnet50_ibn_a_NO_AUG",\
                  "resnet101_ibn_b_NO_AUG", "resnet101_ibn_a_NO_AUG",\
                  "resnet50_only_FLIP_ROTAT_ONLY", "resnet50_only_NO_AUG",\
                 "resnet101_only_FLIP_ROTAT_ONLY", "resnet101_only_NO_AUG",
                 ]
'''
IBN_model_nameList = ["resnet18_only_NO_AUG",\
                 "resnet34_ibn_a_FLIP_ROTAT_ONLY"]


for IBN_model_name in IBN_model_nameList:
        # set specific model to load
    if "resnet50_ibn_a" in IBN_model_name:
        IBN_model_url = "https://github.com/XingangPan/IBN-Net/releases/download/v1.0/resnet50_ibn_a-d9d0bb7b.pth"
        model_real_name = "resnet50_ibn_a"
        
    elif "resnet50_ibn_b" in IBN_model_name:
        IBN_model_url = "https://github.com/XingangPan/IBN-Net/releases/download/v1.0/resnet50_ibn_b-9ca61e85.pth"
        model_real_name = "resnet50_ibn_b"
        
    elif "resnet101_ibn_a" in IBN_model_name:
        IBN_model_url = "https://github.com/XingangPan/IBN-Net/releases/download/v1.0/resnet101_ibn_a-59ea0ac6.pth"
        model_real_name = "resnet101_ibn_a"
        
    elif "resnet101_ibn_b" in IBN_model_name:
        IBN_model_url = "https://github.com/XingangPan/IBN-Net/releases/download/v1.0/resnet101_ibn_b-c55f6dba.pth"
        model_real_name = "resnet101_ibn_b"
        
    elif "resnet18_ibn_a" in IBN_model_name:
        IBN_model_url = "https://github.com/XingangPan/IBN-Net/releases/download/v1.0/resnet18_ibn_a-2f571257.pth"
        model_real_name = "resnet18_ibn_a"
        
    elif "resnet18_ibn_b" in IBN_model_name:
        IBN_model_url = "https://github.com/XingangPan/IBN-Net/releases/download/v1.0/resnet18_ibn_b-bc2f3c11.pth"
        model_real_name = "resnet18_ibn_b"
        
    elif "resnet34_ibn_a" in IBN_model_name:
        IBN_model_url = "https://github.com/XingangPan/IBN-Net/releases/download/v1.0/resnet34_ibn_a-94bc1577.pth"
        model_real_name = "resnet34_ibn_a"
        
    elif "resnet34_ibn_b" in IBN_model_name:
        IBN_model_url = "https://github.com/XingangPan/IBN-Net/releases/download/v1.0/resnet34_ibn_b-04134c37.pth"
        model_real_name = "resnet34_ibn_b"
        
    else:
        IBN_model_url = None
        model_real_name = IBN_model_name


    model = get_pretrained_fc_linear(model_real_name, IBN_model_url, True)

        
    # choose either augmented or NOT augmented dataset    
    if "NO_AUG" in IBN_model_name:
        with open(f"{datasetRootPath}/012323_NO_AUG_train_val_and_test_dataloaders_dill.pkl", "rb") as fp:
            dataloaders=dill.load(fp)
    elif "FLIP" in IBN_model_name:
        with open(f"{datasetRootPath}/012323_FLIP_ONLY_train_val_and_test_dataloaders_dill.pkl", "rb") as fp:
            dataloaders=dill.load(fp)
        
        
        
    # train the model and save to the list
    num_epochs=99


    criterion = torch.nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    print(f"Now processing {IBN_model_name}")
    pretrained_ResNet18_metrics, pretrained_ResNet18_model = train_model(model, dataloaders['train'], dataloaders['val'], criterion=criterion, \
                                                                         optimizer=optimizer, num_epochs=num_epochs, model_name=IBN_model_name)

    modelList.append((IBN_model_name, pretrained_ResNet18_model))
    metricsList.append((IBN_model_name, pretrained_ResNet18_metrics))
    with open(f"{datasetRootPath}/012423_modelList_and_metircsList.pkl", "wb") as f:
      dill.dump([modelList, metricsList], f, recurse=True)
    print(f"Finish saving modelList and metricsList till {IBN_model_name}")
    
    
print("FINISHED ALL PROCESSING!")

### Test images

In [ ]:
for (name, model) in modelList:
    print(f"Now predicting on {name}")
    model.eval()
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    sigm = torch.nn.Sigmoid()

    y_pred = []
    y_true = []
    y_names = []

    with torch.no_grad():
    # iterate over test data
        for inputs, labels, names in dataloaders['test']:
            inputs = inputs.to(device)
            labels = labels.to(device)
            y_names.extend(names)

            outputs = model(inputs) # Feed Network
            preds = sigm(outputs).cpu().reshape(-1).detach().numpy().round()

            y_pred.extend(preds) # Save Prediction
            y_true.extend(labels.cpu()) # Save Truth
    test_df = pd.DataFrame(zip(y_pred, y_true, y_names), columns=["y_pred", "y_true", "fname"])
    #print(test_df.head)
    print(get_metrics (y_true, y_pred))
    
    with open(f"{datasetRootPath}/{name}_trained_model_and_test_results_dill.pkl", "wb") as f:
      dill.dump([model, test_df], f, recurse=True)
    print("Finished saving the results!")
    
print("Finished all predictions!")

### Get test statistics on wrongly classified test images

In [ ]:
# Get list of all pkl files
list_all_pkl_files = glob.glob(f"{datasetRootPath}/*trained_model_and_test_results*pkl")
list_all_pkl_files

In [ ]:
FP_dict = dict()
FN_dict = dict()
final_pd = pd.DataFrame()
count = 0
for pkl_path in list_all_pkl_files:
    
    name = pkl_path.split("/")[-1].split("_trained_model_and_test_results_dill.pkl")[0]
    print(f"Working on {name}")
    try:
      with open(f"{pkl_path}", "rb") as f:
          model, test_df = dill.load(f)
    except:
      continue
    count += 1
    #print(test_df.sort_values("fname"))
    new_test_df = test_df.rename({"y_pred":f"{name}_y_pred"},axis=1)
    if final_pd.shape[0]==0:
        
        final_pd = new_test_df.set_index("fname")[["y_true", f"{name}_y_pred"]]
        
    else:
        final_pd = final_pd.join(new_test_df[["fname", f"{name}_y_pred"]].set_index("fname"))
    if len(FP_dict) == 0:
        FP_dict = dict(zip(test_df.fname, [0]*len(test_df.fname)))
        FN_dict = dict(zip(test_df.fname, [0]*len(test_df.fname)))
    #make sure the test dataset is the same for all the experiments
    assert len((set(test_df.fname)-set(FP_dict.keys())))==0
    
    for index, row in test_df.iterrows():
        # False positive case
        if row.y_pred > row.y_true:
            FP_dict[row.fname]+=1
        # False negative case
        elif row.y_pred < row.y_true:
            FN_dict[row.fname]+=1
            

# build the final Dataframe combined FP and FN rates
FP_dict = pd.DataFrame(list(sorted(FP_dict.items(), key=lambda item: -item[1])), columns=["fname", f"Num_FP_in_total_{count}"])
FN_dict = pd.DataFrame(list(sorted(FN_dict.items(), key=lambda item: -item[1])), columns=["fname", f"Num_FN_in_total_{count}"])
            
print(f"Top 10 FPs:\n {FP_dict.head(10)}")
print(f"Top 10 FNs:\n {FN_dict.head(10)}")

        
 

In [ ]:
final_pd.loc["SOB_B_F-14-25197-400-042.png",].sort_values(ascending=True)

In [ ]:
FN_dict.to_csv("012523_num_FNs_all_400x_test_images.csv", index=False)
FP_dict.to_csv("012523_num_FPs_all_400x_test_images.csv", index=False)
final_pd.to_csv("012523_all_predictions_all_400x_test_iamges.csv")